In [ ]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Dataset.py

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import os

import torch
from torch.utils.data import Dataset

import random
import math
import re
from PIL import Image, ImageOps, ImageEnhance
import PIL
import numpy as np

import torch
import torch.nn as nn
import torchvision.transforms as transforms


class SkinDataset(Dataset):
    def __init__(self, root, mode, transform=None):
        self.root = root
        self.mode = mode
        assert self.mode in ["train", "valid", "test"]
        self.transform = transform
        csv_file = os.path.join(root, "ISIC_2019_Training_GroundTruth.csv")
        self.file = pd.read_csv(csv_file)

        self.images = self.file["image"].values
        self.labels = self.file.iloc[:, 1:].values.astype("int")
        self.targets = np.argmax(self.labels, axis=1)

        initial_len = len(self.images)

        # data split
        np.random.seed(0)
        idxs = np.random.permutation(initial_len)
        self.images = self.images[idxs]
        self.targets = self.targets[idxs]

        if self.mode == "train":
            self.images = self.images[:int(0.7*initial_len)]
            self.targets = self.targets[:int(0.7*initial_len)]
        elif self.mode == "valid":
            self.images = self.images[int(
                0.7*initial_len):int(0.8*initial_len)]
            self.targets = self.targets[int(
                0.7*initial_len):int(0.8*initial_len)]
        else:
            self.images = self.images[int(0.8*initial_len):]
            self.targets = self.targets[int(0.8*initial_len):]

        self.n_classes = len(np.unique(self.targets))
        assert self.n_classes == 8

    def __getitem__(self, index):
        """
        Args:
            index: the index of item
        Returns:
            image and its labels
        """
        image_name = os.path.join(
            self.root, "ISIC_2019_Training_Input", self.images[index] + ".jpg")
        img = Image.open(image_name).convert("RGB")
        label = self.targets[index]
        if self.transform is not None:
            if not isinstance(self.transform, list):
                img = self.transform(img)
            else:
                img0 = self.transform[0](img)
                img1 = self.transform[1](img)
                img = [img0, img1]
        return img, label

    def __len__(self):
        return len(self.images)


class ichDataset(Dataset):
    def __init__(self, root, mode, transform=None):
        self.root = root
        self.mode = mode
        assert self.mode in ["train", "valid", "test"]
        self.transform = transform
        self.data = pd.read_csv(os.path.join(self.root, "data.csv"))
        self.image_id = self.data["id"].to_numpy()
        self.targets = self.data["class"].to_numpy()
        initial_len = len(self.targets)

        # split
        np.random.seed(0)
        idxs = np.random.permutation(len(self.targets))
        self.image_id = self.image_id[idxs]
        self.targets = self.targets[idxs]

        if self.mode == "train":
            self.image_id = self.image_id[:int(0.7*initial_len)]
            self.targets = self.targets[:int(0.7*initial_len)]
        elif self.mode == "valid":
            self.image_id = self.image_id[int(0.7*initial_len):int(0.8*initial_len)]
            self.targets = self.targets[int(0.7*initial_len):int(0.8*initial_len)]
        else:
            self.image_id = self.image_id[int(0.8*initial_len):]
            self.targets = self.targets[int(0.8*initial_len):]

        self.n_classes = len(np.unique(self.targets))
        assert self.n_classes == 5

    def __getitem__(self, index):
        id, target = self.image_id[index], self.targets[index]
        img = self.read_image(id)

        if self.transform is not None:
            if not isinstance(self.transform, list):
                img = self.transform(img)
            else:
                img0 = self.transform[0](img)
                img1 = self.transform[1](img)
                img = [img0, img1]
        return img, target

    def __len__(self):
        return len(self.targets)

    def read_image(self, id):
        image_path = os.path.join(self.root, "stage_1_train_images", id+".png")
        image = Image.open(image_path).convert("RGB")
        return image

Randaugment.py

In [ ]:

_PIL_VER = tuple([int(x) for x in PIL.__version__.split('.')[:2]])

_FILL = (128, 128, 128)

# This signifies the max integer that the controller RNN could predict for the
# augmentation scheme.
_MAX_LEVEL = 10.

_HPARAMS_DEFAULT = dict(
    translate_const=250,
    img_mean=_FILL,
)

_RANDOM_INTERPOLATION = (Image.BILINEAR, Image.BICUBIC)


def _interpolation(kwargs):
    interpolation = kwargs.pop('resample', Image.BILINEAR)
    if isinstance(interpolation, (list, tuple)):
        return random.choice(interpolation)
    else:
        return interpolation


def _check_args_tf(kwargs):
    if 'fillcolor' in kwargs and _PIL_VER < (5, 0):
        kwargs.pop('fillcolor')
    kwargs['resample'] = _interpolation(kwargs)


def shear_x(img, factor, **kwargs):
    _check_args_tf(kwargs)
    return img.transform(img.size, Image.AFFINE, (1, factor, 0, 0, 1, 0), **kwargs)


def shear_y(img, factor, **kwargs):
    _check_args_tf(kwargs)
    return img.transform(img.size, Image.AFFINE, (1, 0, 0, factor, 1, 0), **kwargs)


def translate_x_rel(img, pct, **kwargs):
    pixels = pct * img.size[0]
    _check_args_tf(kwargs)
    return img.transform(img.size, Image.AFFINE, (1, 0, pixels, 0, 1, 0), **kwargs)


def translate_y_rel(img, pct, **kwargs):
    pixels = pct * img.size[1]
    _check_args_tf(kwargs)
    return img.transform(img.size, Image.AFFINE, (1, 0, 0, 0, 1, pixels), **kwargs)


def translate_x_abs(img, pixels, **kwargs):
    _check_args_tf(kwargs)
    return img.transform(img.size, Image.AFFINE, (1, 0, pixels, 0, 1, 0), **kwargs)


def translate_y_abs(img, pixels, **kwargs):
    _check_args_tf(kwargs)
    return img.transform(img.size, Image.AFFINE, (1, 0, 0, 0, 1, pixels), **kwargs)


def rotate(img, degrees, **kwargs):
    _check_args_tf(kwargs)
    if _PIL_VER >= (5, 2):
        return img.rotate(degrees, **kwargs)
    elif _PIL_VER >= (5, 0):
        w, h = img.size
        post_trans = (0, 0)
        rotn_center = (w / 2.0, h / 2.0)
        angle = -math.radians(degrees)
        matrix = [
            round(math.cos(angle), 15),
            round(math.sin(angle), 15),
            0.0,
            round(-math.sin(angle), 15),
            round(math.cos(angle), 15),
            0.0,
        ]

        def transform(x, y, matrix):
            (a, b, c, d, e, f) = matrix
            return a * x + b * y + c, d * x + e * y + f

        matrix[2], matrix[5] = transform(
            -rotn_center[0] - post_trans[0], -rotn_center[1] - post_trans[1], matrix
        )
        matrix[2] += rotn_center[0]
        matrix[5] += rotn_center[1]
        return img.transform(img.size, Image.AFFINE, matrix, **kwargs)
    else:
        return img.rotate(degrees, resample=kwargs['resample'])


def auto_contrast(img, **__):
    return ImageOps.autocontrast(img)


def invert(img, **__):
    return ImageOps.invert(img)


def identity(img, **__):
    return img


def equalize(img, **__):
    return ImageOps.equalize(img)


def solarize(img, thresh, **__):
    return ImageOps.solarize(img, thresh)


def solarize_add(img, add, thresh=128, **__):
    lut = []
    for i in range(256):
        if i < thresh:
            lut.append(min(255, i + add))
        else:
            lut.append(i)
    if img.mode in ("L", "RGB"):
        if img.mode == "RGB" and len(lut) == 256:
            lut = lut + lut + lut
        return img.point(lut)
    else:
        return img


def posterize(img, bits_to_keep, **__):
    if bits_to_keep >= 8:
        return img
    return ImageOps.posterize(img, bits_to_keep)


def contrast(img, factor, **__):
    return ImageEnhance.Contrast(img).enhance(factor)


def color(img, factor, **__):
    return ImageEnhance.Color(img).enhance(factor)


def brightness(img, factor, **__):
    return ImageEnhance.Brightness(img).enhance(factor)


def sharpness(img, factor, **__):
    return ImageEnhance.Sharpness(img).enhance(factor)


def _randomly_negate(v):
    """With 50% prob, negate the value"""
    return -v if random.random() > 0.5 else v


def _rotate_level_to_arg(level, _hparams):
    # range [-30, 30]
    level = (level / _MAX_LEVEL) * 30.
    level = _randomly_negate(level)
    return level,


def _enhance_level_to_arg(level, _hparams):
    # range [0.1, 1.9]
    return (level / _MAX_LEVEL) * 1.8 + 0.1,


def _shear_level_to_arg(level, _hparams):
    # range [-0.3, 0.3]
    level = (level / _MAX_LEVEL) * 0.3
    level = _randomly_negate(level)
    return level,


def _translate_abs_level_to_arg(level, hparams):
    translate_const = hparams['translate_const']
    level = (level / _MAX_LEVEL) * float(translate_const)
    level = _randomly_negate(level)
    return level,


def _translate_rel_level_to_arg(level, _hparams):
    # range [-0.45, 0.45]
    level = (level / _MAX_LEVEL) * 0.45
    level = _randomly_negate(level)
    return level,


def _posterize_original_level_to_arg(level, _hparams):
    # As per original AutoAugment paper description
    # range [4, 8], 'keep 4 up to 8 MSB of image'
    return int((level / _MAX_LEVEL) * 4) + 4,


def _posterize_research_level_to_arg(level, _hparams):
    # As per Tensorflow models research and UDA impl
    # range [4, 0], 'keep 4 down to 0 MSB of original image'
    return 4 - int((level / _MAX_LEVEL) * 4),


def _posterize_tpu_level_to_arg(level, _hparams):
    # As per Tensorflow TPU EfficientNet impl
    # range [0, 4], 'keep 0 up to 4 MSB of original image'
    return int((level / _MAX_LEVEL) * 4),


def _solarize_level_to_arg(level, _hparams):
    # range [0, 256]
    return int((level / _MAX_LEVEL) * 256),


def _solarize_add_level_to_arg(level, _hparams):
    # range [0, 110]
    return int((level / _MAX_LEVEL) * 110),


LEVEL_TO_ARG = {
    'AutoContrast': None,
    'Equalize': None,
    'Invert': None,
    'Identity': None,
    'Rotate': _rotate_level_to_arg,
    # There are several variations of the posterize level scaling in various Tensorflow/Google repositories/papers
    'PosterizeOriginal': _posterize_original_level_to_arg,
    'PosterizeResearch': _posterize_research_level_to_arg,
    'PosterizeTpu': _posterize_tpu_level_to_arg,
    'Solarize': _solarize_level_to_arg,
    'SolarizeAdd': _solarize_add_level_to_arg,
    'Color': _enhance_level_to_arg,
    'Contrast': _enhance_level_to_arg,
    'Brightness': _enhance_level_to_arg,
    'Sharpness': _enhance_level_to_arg,
    'ShearX': _shear_level_to_arg,
    'ShearY': _shear_level_to_arg,
    'TranslateX': _translate_abs_level_to_arg,
    'TranslateY': _translate_abs_level_to_arg,
    'TranslateXRel': _translate_rel_level_to_arg,
    'TranslateYRel': _translate_rel_level_to_arg,
}


NAME_TO_OP = {
    'AutoContrast': auto_contrast,
    'Equalize': equalize,
    'Invert': invert,
    'Identity': identity,
    'Rotate': rotate,
    'PosterizeOriginal': posterize,
    'PosterizeResearch': posterize,
    'PosterizeTpu': posterize,
    'Solarize': solarize,
    'SolarizeAdd': solarize_add,
    'Color': color,
    'Contrast': contrast,
    'Brightness': brightness,
    'Sharpness': sharpness,
    'ShearX': shear_x,
    'ShearY': shear_y,
    'TranslateX': translate_x_abs,
    'TranslateY': translate_y_abs,
    'TranslateXRel': translate_x_rel,
    'TranslateYRel': translate_y_rel,
}


class AutoAugmentOp:

    def __init__(self, name, prob=0.5, magnitude=10, hparams=None):
        hparams = hparams or _HPARAMS_DEFAULT
        self.aug_fn = NAME_TO_OP[name]
        self.level_fn = LEVEL_TO_ARG[name]
        self.prob = prob
        self.magnitude = magnitude
        self.hparams = hparams.copy()
        self.kwargs = dict(
            fillcolor=hparams['img_mean'] if 'img_mean' in hparams else _FILL,
            resample=hparams['interpolation'] if 'interpolation' in hparams else _RANDOM_INTERPOLATION,
        )

        # If magnitude_std is > 0, we introduce some randomness
        # in the usually fixed policy and sample magnitude from a normal distribution
        # with mean `magnitude` and std-dev of `magnitude_std`.
        # NOTE This is my own hack, being tested, not in papers or reference impls.
        self.magnitude_std = self.hparams.get('magnitude_std', 0)

    def __call__(self, img):
        if random.random() > self.prob:
            return img
        magnitude = self.magnitude
        if self.magnitude_std and self.magnitude_std > 0:
            magnitude = random.gauss(magnitude, self.magnitude_std)
        magnitude = min(_MAX_LEVEL, max(0, magnitude)) # clip to valid range
        level_args = self.level_fn(magnitude, self.hparams) if self.level_fn is not None else tuple()
        return self.aug_fn(img, *level_args, **self.kwargs)


_RAND_TRANSFORMS = [
    'AutoContrast',
    'Equalize',
    'Invert',
    'Rotate',
    'PosterizeTpu',
    'Solarize',
    'SolarizeAdd',
    'Color',
    'Contrast',
    'Brightness',
    'Sharpness',
    'ShearX',
    'ShearY',
    'TranslateXRel',
    'TranslateYRel',
    #'Cutout'  # FIXME I implement this as random erasing separately
]

_RAND_TRANSFORMS_CMC = [
    'AutoContrast',
    'Identity',
    'Rotate',
    'Sharpness',
    'ShearX',
    'ShearY',
    'TranslateXRel',
    'TranslateYRel',
    #'Cutout'  # FIXME I implement this as random erasing separately
]


# These experimental weights are based loosely on the relative improvements mentioned in paper.
# They may not result in increased performance, but could likely be tuned to so.
_RAND_CHOICE_WEIGHTS_0 = {
    'Rotate': 0.3,
    'ShearX': 0.2,
    'ShearY': 0.2,
    'TranslateXRel': 0.1,
    'TranslateYRel': 0.1,
    'Color': .025,
    'Sharpness': 0.025,
    'AutoContrast': 0.025,
    'Solarize': .005,
    'SolarizeAdd': .005,
    'Contrast': .005,
    'Brightness': .005,
    'Equalize': .005,
    'PosterizeTpu': 0,
    'Invert': 0,
}


def _select_rand_weights(weight_idx=0, transforms=None):
    transforms = transforms or _RAND_TRANSFORMS
    assert weight_idx == 0  # only one set of weights currently
    rand_weights = _RAND_CHOICE_WEIGHTS_0
    probs = [rand_weights[k] for k in transforms]
    probs /= np.sum(probs)
    return probs


def rand_augment_ops(magnitude=10, hparams=None, transforms=None):
    """rand augment ops for RGB images"""
    hparams = hparams or _HPARAMS_DEFAULT
    transforms = transforms or _RAND_TRANSFORMS
    return [AutoAugmentOp(
        name, prob=0.5, magnitude=magnitude, hparams=hparams) for name in transforms]


def rand_augment_ops_cmc(magnitude=10, hparams=None, transforms=None):
    """rand augment ops for CMC images (removing color ops)"""
    hparams = hparams or _HPARAMS_DEFAULT
    transforms = transforms or _RAND_TRANSFORMS_CMC
    return [AutoAugmentOp(
        name, prob=0.5, magnitude=magnitude, hparams=hparams) for name in transforms]


class RandAugment:
    def __init__(self, ops, num_layers=2, choice_weights=None):
        self.ops = ops
        self.num_layers = num_layers
        self.choice_weights = choice_weights

    def __call__(self, img):
        # no replacement when using weighted choice
        ops = np.random.choice(
            self.ops, self.num_layers, replace=self.choice_weights is None, p=self.choice_weights)
        for op in ops:
            img = op(img)
        return img


def rand_augment_transform(config_str, hparams, use_cmc=False):
    """
    Create a RandAugment transform
    :param config_str: String defining configuration of random augmentation. Consists of multiple sections separated by
    dashes ('-'). The first section defines the specific variant of rand augment (currently only 'rand'). The remaining
    sections, not order sepecific determine
        'm' - integer magnitude of rand augment
        'n' - integer num layers (number of transform ops selected per image)
        'w' - integer probabiliy weight index (index of a set of weights to influence choice of op)
        'mstd' -  float std deviation of magnitude noise applied
    Ex 'rand-m9-n3-mstd0.5' results in RandAugment with magnitude 9, num_layers 3, magnitude_std 0.5
    'rand-mstd1-w0' results in magnitude_std 1.0, weights 0, default magnitude of 10 and num_layers 2
    :param hparams: Other hparams (kwargs) for the RandAugmentation scheme
    :param use_cmc: Flag indicates removing augmentation for coloring ops.
    :return: A PyTorch compatible Transform
    """
    magnitude = _MAX_LEVEL  # default to _MAX_LEVEL for magnitude (currently 10)
    num_layers = 2  # default to 2 ops per image
    weight_idx = None  # default to no probability weights for op choice
    config = config_str.split('-')
    assert config[0] == 'rand'
    config = config[1:]
    for c in config:
        cs = re.split(r'(\d.*)', c)
        if len(cs) < 2:
            continue
        key, val = cs[:2]
        if key == 'mstd':
            # noise param injected via hparams for now
            hparams.setdefault('magnitude_std', float(val))
        elif key == 'm':
            magnitude = int(val)
        elif key == 'n':
            num_layers = int(val)
        elif key == 'w':
            weight_idx = int(val)
        else:
            assert False, 'Unknown RandAugment config section'
    if use_cmc:
        ra_ops = rand_augment_ops_cmc(magnitude=magnitude, hparams=hparams)
    else:
        ra_ops = rand_augment_ops(magnitude=magnitude, hparams=hparams)
    choice_weights = None if weight_idx is None else _select_rand_weights(weight_idx)
    return RandAugment(ra_ops, num_layers, choice_weights=choice_weights)


class GaussianBlur(object):
    """blur a single image on CPU"""
    def __init__(self, kernel_size):
        radias = kernel_size // 2
        kernel_size = radias * 2 + 1
        self.blur_h = nn.Conv2d(3, 3, kernel_size=(kernel_size, 1),
                                stride=1, padding=0, bias=False, groups=3)
        self.blur_v = nn.Conv2d(3, 3, kernel_size=(1, kernel_size),
                                stride=1, padding=0, bias=False, groups=3)
        self.k = kernel_size
        self.r = radias

        self.blur = nn.Sequential(
            nn.ReflectionPad2d(radias),
            self.blur_h,
            self.blur_v
        )

        self.pil_to_tensor = transforms.ToTensor()
        self.tensor_to_pil = transforms.ToPILImage()

    def __call__(self, img):
        img = self.pil_to_tensor(img).unsqueeze(0)

        sigma = np.random.uniform(0.1, 2.0)
        x = np.arange(-self.r, self.r + 1)
        x = np.exp(-np.power(x, 2) / (2 * sigma * sigma))
        x = x / x.sum()
        x = torch.from_numpy(x).view(1, -1).repeat(3, 1)

        self.blur_h.weight.data.copy_(x.view(3, 1, self.k, 1))
        self.blur_v.weight.data.copy_(x.view(3, 1, 1, self.k))

        with torch.no_grad():
            img = self.blur(img)
            img = img.squeeze()

        img = self.tensor_to_pil(img)

        return img

Get_dataset

In [ ]:
import logging
from collections import Counter
from torchvision.transforms import transforms

#from dataset.dataset import SkinDataset, ichDataset
#from dataset.randaugment import rand_augment_transform


def get_datasets(args):
   if args.dataset == "isic2019":
      root = "/content/drive/MyDrive/"
      normalize = transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
      rgb_mean = (0.485, 0.456, 0.406)
      ra_params = dict(translate_const=int(224 * 0.45), img_mean=tuple([min(255, round(255 * x)) for x in rgb_mean]), )
      trans = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.08, 1.)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.4, 0.0)
            ], p=1.0),
            rand_augment_transform(
                'rand-n{}-m{}-mstd0.5'.format(2, 10), ra_params),
            transforms.ToTensor(),
            normalize,
        ])
      augmentation_sim = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
            normalize
        ])
      val_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize
        ])

      train_dataset = SkinDataset(root=root, mode="train",
                                    transform=[trans, augmentation_sim])
      val_dataset = SkinDataset(root=root, mode="valid",
                                  transform=val_transform)
      test_dataset = SkinDataset(root=root, mode="test",
                                   transform=val_transform)

    # elif args.dataset == "ich":
    #     root = "/home/wnn/dataset/ICH"
    #     normalize = transforms.Normalize([0.485, 0.456, 0.406],
    #                                      [0.229, 0.224, 0.225])

    #     trans1 = transforms.Compose([
    #         transforms.Resize((224, 224)),
    #         transforms.RandomAffine(degrees=10, translate=(0.02, 0.02)),
    #         transforms.RandomHorizontalFlip(),
    #         transforms.ToTensor(),
    #         normalize,
    #     ])
    #     trans2 = transforms.Compose([
    #         transforms.Resize((224, 224)),
    #         transforms.RandomAffine(degrees=10, translate=(0.02, 0.02)),
    #         transforms.RandomHorizontalFlip(),
    #         transforms.ToTensor(),
    #         normalize,
    #     ])
    #     val_transform = transforms.Compose([
    #         transforms.Resize((224, 224)),
    #         transforms.ToTensor(),
    #         normalize
    #     ])

    #     train_dataset = ichDataset(
    #         root=root, mode="train", transform=[trans1, trans2])
    #     val_dataset = ichDataset(
    #         root=root, mode="valid", transform=val_transform)
    #     test_dataset = ichDataset(
    #         root=root, mode="test", transform=val_transform)

    # else:
    #     raise

    # logging.info(Counter(train_dataset.targets))
    # logging.info(Counter(val_dataset.targets))
    # logging.info(Counter(test_dataset.targets))

      return train_dataset, val_dataset, test_dataset


FedAvg.py

In [ ]:
import torch
from torch import nn
import numpy as np
import copy


def FedAvg(w, dict_len):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        w_avg[k] = w_avg[k] * dict_len[0]
        for i in range(1, len(w)):
            w_avg[k] += w[i][k] * dict_len[i]
        w_avg[k] = w_avg[k] / sum(dict_len)
    return w_avg

local training.py

In [ ]:
import logging
import numpy as np
import copy

import torch
import torch.optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

    def get_num_class_list(self):
        self.n_classes = self.dataset.n_classes
        class_num = np.array([0] * self.n_classes)
        for idx in self.idxs:
            label = self.dataset.targets[idx]
            class_num[label] += 1
        return class_num.tolist()


class LocalUpdate(object):
    def __init__(self, args, id, dataset, idxs):
        self.args = args
        self.id = id
        self.local_dataset = DatasetSplit(dataset, idxs)
        self.class_num_list = self.local_dataset.get_num_class_list()
        logging.info(
            f"Client{id} ===> Each class num: {self.class_num_list}, Total: {len(self.local_dataset)}")
        self.ldr_train = DataLoader(
            self.local_dataset, batch_size=self.args.batch_size, shuffle=True, num_workers=4)
        self.epoch = 0
        self.iter_num = 0
        self.lr = self.args.base_lr


    def train(self, net, writer):
        net.train()
        # set the optimizer
        self.optimizer = torch.optim.Adam(
            net.parameters(), lr=self.lr, betas=(0.9, 0.999), weight_decay=5e-4)
        print(f"Id: {self.id}, Num: {len(self.local_dataset)}")

        # train and update
        epoch_loss = []
        ce_criterion = nn.CrossEntropyLoss()
        for epoch in range(self.args.local_ep):
            batch_loss = []
            for (images, labels) in self.ldr_train:
                images, labels = images.cuda(), labels.cuda()

                _, logits = net(images)
                loss = ce_criterion(logits, labels)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                batch_loss.append(loss.item())
                writer.add_scalar(
                    f'client{self.id}/loss_train', loss.item(), self.iter_num)
                self.iter_num += 1
            self.epoch = self.epoch + 1
            epoch_loss.append(np.array(batch_loss).mean())

        return net.state_dict(), np.array(epoch_loss).mean()


    def train_FedIIC(self, net, prototypes, writer):
        net.train()
        # set the optimizer
        self.optimizer = torch.optim.Adam(
            net.parameters(), lr=self.lr, betas=(0.9, 0.999), weight_decay=5e-4)
        print(f"Id: {self.id}, Num: {len(self.local_dataset)}")

        prototypes = F.normalize(prototypes, dim=1).detach().clone().cuda()

        intra_cl_criterion = IntraSCL(cls_num_list=self.class_num_list)
        inter_cl_criterion = InterSCL(cls_num_list=self.class_num_list)
        ce_criterion = DALA(
            cls_num_list=self.class_num_list, cls_loss=self.loss_class, epoch=self.epoch, args=self.args)

        # train and update
        epoch_loss = []
        for epoch in range(self.args.local_ep):
            batch_loss = []
            for (images, labels) in self.ldr_train:
                assert isinstance(images, list)
                images = torch.cat([images[0], images[1]], dim=0)
                images, labels = images.cuda(), labels.cuda()

                features, logits = net(images, project=True)

                features = F.normalize(features, dim=1)

                split_size = features.size()[0] // 2

                f1, f2 = torch.split(features, [split_size, split_size], dim=0)
                features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)
                logits, _ = torch.split(
                    logits, [split_size, split_size], dim=0)

                loss_ce = ce_criterion(logits, labels)
                loss_cl_inter = inter_cl_criterion(prototypes, features, labels)
                loss_cl_intra = intra_cl_criterion(features, labels)

                loss = loss_ce + self.args.k1*loss_cl_intra + self.args.k2*loss_cl_inter

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                batch_loss.append(loss.item())
                writer.add_scalar(
                    f'client{self.id}/loss', loss.item(), self.iter_num)
                writer.add_scalar(
                    f'client{self.id}/loss_train', loss_ce, self.iter_num)
                writer.add_scalar(
                    f'client{self.id}/loss_cl_inter', loss_cl_inter, self.iter_num)
                writer.add_scalar(
                    f'client{self.id}/loss_cl_intra', loss_cl_intra, self.iter_num)

                self.iter_num += 1
            self.epoch = self.epoch + 1
            epoch_loss.append(np.array(batch_loss).mean())

        return net.state_dict(), np.array(epoch_loss).mean()

losses.py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class DALA(nn.Module):
    def __init__(self, cls_num_list, cls_loss, epoch, tau=1, weight=None, args=None):
        super(DALA, self).__init__()
        cls_num_list = torch.cuda.FloatTensor(cls_num_list)
        cls_p_list = cls_num_list / cls_num_list.sum()
        cls_loss = cls_loss.cuda()

        t = cls_p_list / (torch.pow(cls_loss, args.d)+1e-5)
        m_list = tau * torch.log(t)

        self.m_list = m_list.view(1, -1)
        self.weight = weight

    def forward(self, x, target):
        x_m = x + self.m_list
        return F.cross_entropy(x_m, target, weight=self.weight)


class IntraSCL(nn.Module):
    def __init__(self, cls_num_list, temperature=0.1):
        super(IntraSCL, self).__init__()
        self.temperature = temperature
        self.cls_num_list = torch.tensor(cls_num_list).cuda()
        self.cls_num_list = self.cls_num_list / torch.sum(self.cls_num_list)

    def forward(self, features, targets):

        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))

        batch_size = features.shape[0]
        targets = targets.contiguous().view(-1, 1)
        targets = targets.repeat(2, 1)
        mask = torch.eq(targets, targets.T).float().to(device)
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * 2).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask

        features = torch.cat(torch.unbind(features, dim=1), dim=0)
        logits = features.mm(features.T)
        temp = self.cls_num_list.gather(0, targets.view(-1)).view(-1, 1)
        temp = temp.mm(temp.T)
        temp = torch.clamp(temp**0.5, min=0.07)
        logits = torch.div(logits, temp)

        # For numerical stability
        logits_max, _ = torch.max(logits, dim=1, keepdim=True)
        logits = logits - logits_max.detach()
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = -1 * mean_log_prob_pos
        loss = loss.view(2, batch_size).mean()

        return loss


class InterSCL(nn.Module):
    def __init__(self, cls_num_list=None, temperature=0.1):
        super(InterSCL, self).__init__()
        self.temperature = temperature
        self.cls_num_list = cls_num_list

    def forward(self, centers1, features, targets):

        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))
        batch_size = features.shape[0]
        targets = targets.contiguous().view(-1, 1)
        targets_centers = torch.arange(
            len(self.cls_num_list), device=device).view(-1, 1)
        targets = torch.cat([targets.repeat(2, 1), targets_centers], dim=0)
        batch_cls_count = torch.eye(len(self.cls_num_list))[
            targets].sum(dim=0).squeeze()

        mask = torch.eq(targets[:2 * batch_size], targets.T).float().to(device)
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * 2).view(-1, 1).to(device),
            0
        )
        logits_mask[:2*batch_size, :2*batch_size] = 0
        mask = mask * logits_mask

        # class-complement
        features = torch.cat(torch.unbind(features, dim=1), dim=0)
        features = torch.cat([features, centers1], dim=0)
        logits = features[:2 * batch_size].mm(features.T)
        logits = torch.div(logits, self.temperature)

        # For numerical stability
        logits_max, _ = torch.max(logits, dim=1, keepdim=True)
        logits = logits - logits_max.detach()
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = -1 * mean_log_prob_pos
        loss = loss.view(2, batch_size).mean()

        return loss

utils.py

In [ ]:
import random
import numpy as np
import torch
from torch.utils.data.dataset import Dataset


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)


def get_class_num(ys):
    from collections import Counter
    num_dict = Counter(ys)
    index = []
    compose = []
    for c in num_dict.keys():
        if num_dict[c] != 0:
            index.append(c)
            compose.append(num_dict[c])
    return index, compose


def classify_label(y, num_classes: int):
    list1 = [[] for _ in range(num_classes)]
    for idx, label in enumerate(y):
        list1[int(label)].append(idx)
    return list1


class TensorDataset(Dataset):
    def __init__(self, images, labels):  # images: n x c x h x w tensor
        self.images = images.detach().float()
        self.labels = labels.detach()

    def __getitem__(self, index):
        return self.images[index], self.labels[index]

    def __len__(self):
        return self.images.shape[0]

sample_dirichlet.py

In [ ]:
import math
import functools
import numpy as np


def clients_indices(list_label2indices: list, num_classes: int, num_clients: int, non_iid_alpha: float, seed=None):
    indices2targets = []
    for label, indices in enumerate(list_label2indices):
        for idx in indices:
            indices2targets.append((idx, label))

    batch_indices = build_non_iid_by_dirichlet(seed=seed,
                                               indices2targets=indices2targets,
                                               non_iid_alpha=non_iid_alpha,
                                               num_classes=num_classes,
                                               num_indices=len(indices2targets),
                                               n_workers=num_clients)
    indices_dirichlet = functools.reduce(lambda x, y: x + y, batch_indices)
    list_client2indices = partition_balance(indices_dirichlet, num_clients)

    return list_client2indices


def partition_balance(idxs, num_split: int):

    num_per_part, r = len(idxs) // num_split, len(idxs) % num_split
    parts = []
    i, r_used = 0, 0
    while i < len(idxs):
        if r_used < r:
            parts.append(idxs[i:(i + num_per_part + 1)])
            i += num_per_part + 1
            r_used += 1
        else:
            parts.append(idxs[i:(i + num_per_part)])
            i += num_per_part

    return parts


def build_non_iid_by_dirichlet(
    seed, indices2targets, non_iid_alpha, num_classes, num_indices, n_workers
):
    random_state = np.random.RandomState(seed)
    n_auxi_workers = 10
    assert n_auxi_workers <= n_workers

    # random shuffle targets indices.
    random_state.shuffle(indices2targets)

    # partition indices.
    from_index = 0
    splitted_targets = []

    num_splits = math.ceil(n_workers / n_auxi_workers)

    split_n_workers = [
        n_auxi_workers
        if idx < num_splits - 1
        else n_workers - n_auxi_workers * (num_splits - 1)
        for idx in range(num_splits)
    ]
    split_ratios = [_n_workers / n_workers for _n_workers in split_n_workers]
    for idx, ratio in enumerate(split_ratios):
        to_index = from_index + int(n_auxi_workers / n_workers * num_indices)
        splitted_targets.append(
            indices2targets[
                from_index: (num_indices if idx == num_splits - 1 else to_index)
            ]
        )
        from_index = to_index

    #
    idx_batch = []
    for _targets in splitted_targets:
        # rebuild _targets.
        _targets = np.array(_targets)
        _targets_size = len(_targets)

        # use auxi_workers for this subset targets.
        _n_workers = min(n_auxi_workers, n_workers)
        #n_workers=10
        n_workers = n_workers - n_auxi_workers

        # get the corresponding idx_batch.
        min_size = 0
        _idx_batch = None
        while min_size < int(0.50 * _targets_size / _n_workers):
            _idx_batch = [[] for _ in range(_n_workers)]
            for _class in range(num_classes):
                # get the corresponding indices in the original 'targets' list.
                idx_class = np.where(_targets[:, 1] == _class)[0]
                idx_class = _targets[idx_class, 0]

                # sampling.
                try:
                    proportions = random_state.dirichlet(
                        np.repeat(non_iid_alpha, _n_workers)
                    )
                    # balance
                    proportions = np.array(
                        [
                            p * (len(idx_j) < _targets_size / _n_workers)
                            for p, idx_j in zip(proportions, _idx_batch)
                        ]
                    )
                    proportions = proportions / proportions.sum()
                    proportions = (np.cumsum(proportions) * len(idx_class)).astype(int)[
                        :-1
                    ]
                    _idx_batch = [
                        idx_j + idx.tolist()
                        for idx_j, idx in zip(
                            _idx_batch, np.split(idx_class, proportions)
                        )
                    ]
                    sizes = [len(idx_j) for idx_j in _idx_batch]
                    min_size = min([_size for _size in sizes])
                except ZeroDivisionError:
                    pass
        if _idx_batch is not None:
            idx_batch += _idx_batch

    return idx_batch

efficient.py

In [ ]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=766c8997eea8e403b634741602af4a6bc2c1ac2fe378983e3c4290e1698ec6fc
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [ ]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.4 MB/s eta 0:00:00


In [ ]:
from __future__ import print_function, division, absolute_import
from efficientnet_pytorch import EfficientNet

pretrained_settings = {
    'efficientnet': {
        'imagenet': {
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
}


def initialize_pretrained_model(model, num_classes, settings):
    assert num_classes == settings['num_classes'],'num_classes should be {}, but is {}'.format(
            settings['num_classes'], num_classes)
    model.input_space = settings['input_space']
    model.input_size = settings['input_size']
    model.input_range = settings['input_range']
    model.mean = settings['mean']
    model.std = settings['std']


def efficientnet_b0(num_classes=1000, pretrained='imagenet'):
    model = EfficientNet.from_pretrained('efficientnet-b0', advprop=False)
    if pretrained is not None:
        settings = pretrained_settings['efficientnet'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model


def efficientnet_b1(num_classes=1000, pretrained='imagenet'):
    model = EfficientNet.from_pretrained('efficientnet-b1', advprop=False)
    if pretrained is not None:
        settings = pretrained_settings['efficientnet'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model


def efficientnet_b2(num_classes=1000, pretrained='imagenet'):
    model = EfficientNet.from_pretrained('efficientnet-b2', advprop=False)
    if pretrained is not None:
        settings = pretrained_settings['efficientnet'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model


def efficientnet_b3(num_classes=1000, pretrained='imagenet'):
    model = EfficientNet.from_pretrained('efficientnet-b3', advprop=False)
    if pretrained is not None:
        settings = pretrained_settings['efficientnet'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model


def efficientnet_b4(num_classes=1000, pretrained='imagenet'):
    model = EfficientNet.from_pretrained('efficientnet-b4', advprop=False)
    if pretrained is not None:
        settings = pretrained_settings['efficientnet'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model


def efficientnet_b5(num_classes=1000, pretrained='imagenet'):
    model = EfficientNet.from_pretrained('efficientnet-b5', advprop=False)
    if pretrained is not None:
        settings = pretrained_settings['efficientnet'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model


def efficientnet_b6(num_classes=1000, pretrained='imagenet'):
    model = EfficientNet.from_pretrained('efficientnet-b6', advprop=False)
    if pretrained is not None:
        settings = pretrained_settings['efficientnet'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model


def efficientnet_b7(num_classes=1000, pretrained='imagenet'):
    model = EfficientNet.from_pretrained('efficientnet-b7', advprop=False)
    if pretrained is not None:
        settings = pretrained_settings['efficientnet'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model


allmodels.py

In [ ]:
!pip install pretrainedmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=2adf43ea4438c71321dbefb426e78357bebad11f7234e47c4ee7031b614fb3b6
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built pretrainedmodels


In [ ]:
import torch.nn as nn
import copy
from torchvision import models
import pretrainedmodels


def get_model(model_name, pretrained=False):
    """Returns a CNN model
    Args:
      model_name: model name
      pretrained: True or False
    Returns:
      model: the desired model
    Raises:
      ValueError: If model name is not recognized.
    """
    if pretrained == False:
        pt = None
    else:
        pt = 'imagenet'

    if model_name == 'Vgg11':
        return models.vgg11(pretrained=pretrained)
    elif model_name == 'Vgg13':
        return models.vgg13(pretrained=pretrained)
    elif model_name == 'Vgg16':
        return models.vgg16(pretrained=pretrained)
    elif model_name == 'Vgg19':
        return models.vgg19(pretrained=pretrained)
    elif model_name == 'Resnet18':
        return models.resnet18(pretrained=pretrained)
    elif model_name == 'Resnet34':
        return models.resnet34(pretrained=pretrained)
    elif model_name == 'Resnet50':
        return models.resnet50(pretrained=pretrained)
    elif model_name == 'Resnet101':
        return models.resnet101(pretrained=pretrained)
    elif model_name == 'Resnet152':
        return models.resnet152(pretrained=pretrained)
    elif model_name == 'Dense121':
        return models.densenet121(pretrained=pretrained)
    elif model_name == 'Dense169':
        return models.densenet169(pretrained=pretrained)
    elif model_name == 'Dense201':
        return models.densenet201(pretrained=pretrained)
    elif model_name == 'Dense161':
        return models.densenet161(pretrained=pretrained)
    elif model_name == 'SENet50':
        return pretrainedmodels.__dict__['se_resnet50'](num_classes=1000, pretrained=pt)
    elif model_name == 'SENet101':
        return pretrainedmodels.__dict__['se_resnet101'](num_classes=1000, pretrained=pt)
    elif model_name == 'SENet152':
        return pretrainedmodels.__dict__['se_resnet152'](num_classes=1000, pretrained=pt)
    elif model_name == 'SENet154':
        return pretrainedmodels.__dict__['senet154'](num_classes=1000, pretrained=pt)
    elif model_name == 'Efficient_b0':
        return efficientnet_b0(num_classes=1000, pretrained=pt)
    elif model_name == 'Efficient_b1':
        return efficientnet_b1(num_classes=1000, pretrained=pt)
    elif model_name == 'Efficient_b2':
        return efficientnet_b2(num_classes=1000, pretrained=pt)
    elif model_name == 'Efficient_b3':
        return efficientnet_b3(num_classes=1000, pretrained=pt)
    elif model_name == 'Efficient_b4':
        return efficientnet_b4(num_classes=1000, pretrained=pt)
    elif model_name == 'Efficient_b5':
        return efficientnet_b5(num_classes=1000, pretrained=pt)
    elif model_name == 'Efficient_b6':
        return efficientnet_b6(num_classes=1000, pretrained=pt)
    elif model_name == 'Efficient_b7':
        return efficientnet_b7(num_classes=1000, pretrained=pt)

    else:
        raise ValueError('Name of model unknown %s' % model_name)


def modify_last_layer(model_name, model, num_classes):
    """modify the last layer of the CNN model to fit the num_classes
    Args:
      model_name: model name
      model: CNN model
      num_classes: class number
    Returns:
      model: the desired model
    """

    if model_name == 'Efficient_b0':
        num_ftrs = model._fc.in_features
        model._fc = classifier(num_ftrs, num_classes)
        last_layer = model._fc

    # print(model)
    return model, last_layer


def classifier(num_features, num_classes):
    last_linear = nn.Linear(num_features, num_classes, bias=True)
    return last_linear


def get_bbn_model(model_name, model):
    """modify the normal CNN model to BBN model. Only supports Resnet and RegNet model.
    Args:
      model_name: model name
      model: CNN model
    Returns:
      model: the desired model
    """
    if 'Resnet' in model_name:
        model.bbn_backbone = nn.Sequential(
            model.conv1,
            model.bn1,
            model.relu,
            model.maxpool,
            model.layer1,
            model.layer2,
            model.layer3
        )
        model.bbn_cb_block = model.layer4
        model.bbn_rb_block = copy.deepcopy(model.bbn_cb_block)
    elif 'RegNet' in model_name:
        if 'DropBlock' in model_name:
            model.bbn_backbone = nn.Sequential(
                model.stem,
                model.s1,
                model.s2,
                model.s3,
                model.s3_drop_block
            )
            model.bbn_cb_block = nn.Sequential(
                model.s4,
                model.s4_drop_block
            )
            model.bbn_drop_out = model.head.dropout
        else:
            model.bbn_backbone = nn.Sequential(
                model.stem,
                model.s1,
                model.s2,
                model.s3
            )
            model.bbn_cb_block = model.s4

        model.bbn_rb_block = copy.deepcopy(model.bbn_cb_block)
    else:
        raise ValueError(
            'Model %s does not support bbn structure.' % model_name)

    return model


def get_feature_length(model_name, model):
    """get the feature length of the last feature layer
    Args:
      model_name: model name
      model: CNN model
    Returns:
      num_ftrs: the feature length of the last feature layer
    """
    if 'Vgg' in model_name:
        num_ftrs = model.classifier._modules['6'].in_features
    elif 'Dense' in model_name:
        num_ftrs = model.classifier.in_features
    elif 'Resnet' in model_name:
        num_ftrs = model.fc.in_features
    elif 'Efficient' in model_name:
        num_ftrs = model._fc.in_features
    elif 'RegNet' in model_name:
        num_ftrs = model.head.fc.in_features
    else:
        num_ftrs = model.last_linear.in_features

    return num_ftrs

networks.py

In [ ]:
import torch
import torch.nn as nn


class efficientb0(nn.Module):
    def __init__(self, n_classes, args=None):
        super(efficientb0, self).__init__()
        self.n_classes = n_classes
        self.args = args
        self.model = get_model("Efficient_b0", pretrained=True)
        self.num_ftrs = self.model._fc.in_features
        self.model._fc = nn.Linear(self.num_ftrs, self.n_classes)
        self.projector = nn.Sequential(
            nn.Linear(self.num_ftrs, self.num_ftrs),
            nn.Linear(self.num_ftrs, 1024)
        )

    def forward(self, inputs, project=False):
        if project == False:
            # Convolution layers
            x = self.model.extract_features(inputs)
            # Pooling and final linear layer
            x = self.model._avg_pooling(x)
            x = x.flatten(start_dim=1)
            x = self.model._dropout(x)
            x = self.model._fc(x)
            return x, x
        else:
            # Convolution layers
            x = self.model.extract_features(inputs)
            # Pooling and final linear layer
            x = self.model._avg_pooling(x)
            x = x.flatten(start_dim=1)
            features = self.projector(x)
            y = self.model._dropout(x)
            y = self.model._fc(y)
            return features, y

val.py

In [ ]:
import torch
import torch.nn as nn
from sklearn.metrics import confusion_matrix, balanced_accuracy_score


def compute_bacc(model, dataloader, get_confusion_matrix, args):
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for (x, label) in dataloader:
            x = x.cuda()
            _, logits = model(x)
            pred = torch.argmax(logits, dim=1)

            all_preds.append(pred.cpu())
            all_labels.append(label)

    all_labels = torch.cat(all_labels).numpy()
    all_preds = torch.cat(all_preds).numpy()

    acc = balanced_accuracy_score(all_labels, all_preds)

    if get_confusion_matrix:
        conf_matrix = confusion_matrix(all_labels, all_preds)

    if get_confusion_matrix:
        return acc, conf_matrix
    else:
        return acc


def compute_loss(model, dataloader):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    loss = 0.
    with torch.no_grad():
        for (x, label) in dataloader:
            if isinstance(x, list):
                x = x[0]
            x, label = x.cuda(), label.cuda()
            _, logits = model(x)
            loss += criterion(logits, label)
    return loss


def compute_loss_of_classes(model, dataloader, n_classes):
    criterion = nn.CrossEntropyLoss(reduction="none")
    model.eval()

    loss_class = torch.zeros(n_classes).float()
    loss_list = []
    label_list = []

    with torch.no_grad():
        for (x, label) in dataloader:
            if isinstance(x, list):
                x = x[0]
            x, label = x.cuda(), label.cuda()
            _, logits = model(x)
            loss = criterion(logits, label)
            loss_list.append(loss)
            label_list.append(label)

    loss_list = torch.cat(loss_list).cpu()
    label_list = torch.cat(label_list).cpu()

    for i in range(n_classes):
        idx = torch.where(label_list==i)[0]
        loss_class[i] = loss_list[idx].sum()

    return loss_class

train.py

In [ ]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.1 MB/s eta 0:00:00


In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-driver-550-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-server-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-headless-390' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-gcc' for glob 'nvidia*'
Note, selecting 'nvidia-headless-418' for glob 'nvidia*'
Note, selecting 'nvidia-headless-430' for glob 'nvidia*'
Note, selecting 'nvidia-headless-435' for glob 'nvidia*'
Note, selecting 'nvidia-headless-440' for glob 'nvidia*'
Note, selecting 'nvidia-headless-450' for glob 'nvidia*'
Note, selecting 'nvidia-headless-455' for glob 'nvidia*'
Note, selecting 'nvidia-headless-460' for glob 'nvidia*'
Note, se

In [ ]:
!apt install -f

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [ ]:
!apt search lib32gcc1
!apt search xorg-video-abi


Sorting... Done
Full Text Search... Done
Sorting... Done
Full Text Search... Done


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2024-03-05 05:30:03--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.nvidia.com/downloads/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2024-03-05 05:30:03--  https://developer.nvidia.com/downloads/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?2JIvqTNEumQQLGDYy7HktLfQdqrH8LUlowwrNnkX5dVYLytlEntSoyathTPRWvRdB7bbvrBDpPYClBMntj0nHVpjx7KCiLlv6dmZS

In [ ]:
!nvcc –version

/bin/bash: line 1: nvcc: command not found


In [ ]:
import os
import sys
import copy
import logging
import argparse

import torch
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter



class Args:
    def __init__(self):
        self.dataset = "isic2019"
        self.exp = "FedIIC"
        self.batch_size = 2
        self.base_lr = 3e-4
        self.alpha = 1.0
        self.k1 = 2.0
        self.k2 = 2.0
        self.d = 0.25
        self.deterministic = 1
        self.seed = 0
        self.gpu = '1'
        self.local_ep = 1
        self.rounds = 200


if __name__ == '__main__':
    args = Args()
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

    # ------------------------------ deterministic or not ------------------------------
    if args.deterministic:
        cudnn.benchmark = False
        cudnn.deterministic = True
        set_seed(args)

    # ------------------------------ output files ------------------------------
    outputs_dir = 'outputs'
    if not os.path.exists(outputs_dir):
        os.mkdir(outputs_dir)
    exp_dir = os.path.join(outputs_dir, args.exp + '_' + '_' + str(args.local_ep) + '_' + str(args.k1) + '_' + str(args.k2))
    if not os.path.exists(exp_dir):
        os.mkdir(exp_dir)
    models_dir = os.path.join(exp_dir, 'models')
    if not os.path.exists(models_dir):
        os.mkdir(models_dir)
    logs_dir = os.path.join(exp_dir, 'logs')
    if not os.path.exists(logs_dir):
        os.mkdir(logs_dir)
    tensorboard_dir = os.path.join(exp_dir, 'tensorboard')
    if not os.path.exists(tensorboard_dir):
        os.mkdir(tensorboard_dir)

    logging.basicConfig(filename=logs_dir+'/logs.txt', level=logging.INFO,
                        format='[%(asctime)s.%(msecs)03d] %(message)s', datefmt='%H:%M:%S')
    logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))
    logging.info(str(args))
    writer = SummaryWriter(tensorboard_dir)

    # ------------------------------ dataset and dataloader ------------------------------
    train_dataset, val_dataset, test_dataset = get_datasets(args)
    val_loader = DataLoader(
        dataset=val_dataset, batch_size=32, shuffle=False, num_workers=4)

    if args.dataset == "isic2019":
        args.n_clients = 10
    elif args.datasets == "ich":
        args.n_clients = 20
    else:
        raise

    # ------------------------------ global and local settings ------------------------------
    n_classes = train_dataset.n_classes
    net_glob = efficientb0(n_classes=n_classes, args=args).cuda()
    net_glob.train()
    w_glob = net_glob.state_dict()
    w_locals = []
    trainer_locals = []
    net_locals = []
    user_id = list(range(args.n_clients))

    # Here, we follow CreFF (https://arxiv.org/abs/2204.13399).
    list_label2indices = classify_label(train_dataset.targets, n_classes)
    dict_users = clients_indices(list_label2indices, n_classes, args.n_clients, args.alpha, args.seed)
    dict_len = [len(dict_users[id]) for id in user_id]

    for id in user_id:
        trainer_locals.append(LocalUpdate(
            args, id, copy.deepcopy(train_dataset), dict_users[id]))
        w_locals.append(copy.deepcopy(w_glob))
        net_locals.append(copy.deepcopy(net_glob).cuda())

    images_all = {}
    labels_all = {}
    for id in user_id: # to compute loss quickly
        local_set = copy.deepcopy(trainer_locals[id].local_dataset)
        images_all[id] = torch.cat([torch.unsqueeze(local_set[i][0][0], dim=0)
                                    for i in range(len(local_set))])
        labels_all[id] = torch.tensor([int(local_set[i][1])
                                       for i in range(len(local_set))]).long()
        print(id, ':', len(images_all[id]), labels_all[id])

    # ------------------------------ begin training ------------------------------
    set_seed(args)
    best_performance = 0.
    lr = args.base_lr
    acc = []
    for com_round in range(args.rounds):
        logging.info(f'\n======================> round: {com_round} <======================')
        loss_locals = []
        writer.add_scalar('train/lr', lr, com_round)

        with torch.no_grad():
            class_embedding = w_glob["model._fc.weight"].detach().clone().cuda()
            feature_avg = net_glob.projector(class_embedding).detach().clone()
        print("similarity before")
        print(torch.matmul(F.normalize(feature_avg, dim=1),
              F.normalize(feature_avg, dim=1).T))
        feature_avg.requires_grad = True
        optimizer_f = torch.optim.SGD([feature_avg], lr=0.1)
        mask = torch.ones((n_classes, n_classes)) - torch.eye((n_classes))
        mask = mask.cuda()
        for i in range(1000):
            feature_avg_n = F.normalize(feature_avg, dim=1)
            cos_sim = torch.matmul(feature_avg_n, feature_avg_n.T)
            cos_sim = ((cos_sim * mask).max(1)[0]).sum()
            optimizer_f.zero_grad()
            cos_sim.backward()
            optimizer_f.step()
        print("similarity after")
        print(torch.matmul(F.normalize(feature_avg, dim=1),
              F.normalize(feature_avg, dim=1).T))

        loss_matrix = torch.zeros(args.n_clients, n_classes)
        class_num = torch.zeros(args.n_clients, n_classes)
        net_glob = net_glob.cuda()
        for id in user_id:
            class_num[id] = torch.tensor(
                trainer_locals[id].local_dataset.get_num_class_list())
            dataset_client = TensorDataset(images_all[id], labels_all[id])
            dataLoader_client = DataLoader(
                dataset_client, batch_size=32, shuffle=False)
            loss_matrix[id] = compute_loss_of_classes(
                net_glob, dataLoader_client, n_classes)
        num = torch.sum(class_num, dim=0, keepdim=True)
        logging.info("class-num of this round")
        logging.info(num)
        loss_matrix = loss_matrix / (1e-5 + num)
        loss_class = torch.sum(loss_matrix, dim=0)
        logging.info("loss of this round")
        logging.info(loss_class)

        # local training
        for id in user_id:
            trainer_locals[id].lr = lr
            local = trainer_locals[id]
            local.loss_class = loss_class
            net_local = net_locals[id]
            w, loss = local.train_FedIIC(copy.deepcopy(
                net_local), copy.deepcopy(feature_avg), writer)
            w_locals[id] = copy.deepcopy(w)
            loss_locals.append(copy.deepcopy(loss))

        # upload and download
        with torch.no_grad():
            w_glob = FedAvg(w_locals, dict_len)
        net_glob.load_state_dict(w_glob)
        for id in user_id:
            net_locals[id].load_state_dict(w_glob)

        # global validation
        net_glob = net_glob.cuda()
        bacc_g, conf_matrix = compute_bacc(
            net_glob, val_loader, get_confusion_matrix=True, args=args)
        writer.add_scalar(
            f'glob/bacc_val', bacc_g, com_round)
        logging.info('global conf_matrix')
        logging.info(conf_matrix)

        # save model
        if bacc_g > best_performance:
            best_performance = bacc_g
            torch.save(net_glob.state_dict(),  models_dir +
                       f'/best_model_{com_round}_{best_performance}.pth')
            torch.save(net_glob.state_dict(),  models_dir+'/best_model.pth')
        logging.info(f'best bacc: {best_performance}, now bacc: {bacc_g}')
        acc.append(bacc_g)

    writer.close()
    logging.info(acc)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 192MB/s]


Loaded pretrained weights for efficientnet-b0


RuntimeError: No CUDA GPUs are available

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. You can use GPU acceleration.")
else:
    print("CUDA is not available. GPU acceleration is not possible.")

CUDA is not available. GPU acceleration is not possible.


In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117